In [ ]:
!pip install tensorflow-text==2.6.0
!pip install -q tf-models-official==2.6.0

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
from tensorflow.keras.optimizers import Adam
tf.get_logger().setLevel('ERROR')

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/games-reviews/games/reviews_train.csv', index_col=0)
test = pd.read_csv('../input/games-reviews/games/reviews_test.csv', index_col=0)

In [ ]:
train = train[train["review"] != 0]

In [ ]:
train.head()

In [ ]:
train = train.dropna(how='any',axis=0)
train

In [ ]:
test = test.fillna('')

In [ ]:
train.loc[train.like == -1, 'like'] = 0
train

In [ ]:
y_train = train['like']
X_train = train.drop(['like'], axis=1)

In [ ]:
y_train_c = y_train
# y_train_c = tf.keras.utils.to_categorical(
#     y_train.astype('category').cat.codes.values, num_classes=2)

In [ ]:
X_train

In [ ]:
tfhub_handle_encoder = \
    "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1"
tfhub_handle_preprocess = \
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

In [ ]:
EPOCHS = 1
SAVE_PERIOD = 10
BATCH_SIZE = 64
STEPS_PER_EPOCH_ = len(y_train_c) / BATCH_SIZE

In [ ]:
epochs = 1
# steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
steps_per_epoch = len(y_train_c)
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='review')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
classifier_model = build_classifier_model()

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [ ]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [ ]:
# Create a callback that saves the model's weights every 10 epochs
checkpoint_path = './'
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq= int(SAVE_PERIOD * STEPS_PER_EPOCH_))

In [ ]:
history = classifier_model.fit(
    x=X_train['review'].values,
    y=y_train_c.values,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
#     steps_per_epoch=steps_per_epoch,
    callbacks=[cp_callback],
)

In [ ]:
y_proba_bert = classifier_model.predict(test['review'].values) # tf.sigmoid

In [ ]:
y_proba_bert_ = tf.sigmoid(y_proba_bert)

In [ ]:
predictions = tf.where(y_proba_bert_ < 0.5, -1, 1)

In [ ]:
predictions

In [ ]:
predictions = predictions.numpy()

In [ ]:
sumbit = pd.DataFrame(predictions)

In [ ]:
sumbit = sumbit.rename(columns={0:'like'})

In [ ]:
sumbit

In [ ]:
sumbit.to_csv('task13.csv', index=False)